# Les méthodes d’évaluation d’un modèle prédictif

Après des heures à paramétrer au mieux un modèle d’apprentissage avec la certitude d’avoir écarté les biais qui pourraient orienter les résultats – rappelons qu’un mauvais modèle peut fournir de très mauvais résultats avec une précision étonnante –, les premières prédictions sortent de la machine et nous souhaitons évaluer leur qualité afin de le passer en production ou non.

Bien entendu, le cas présenté plus haut ne vaut que pour sa généralité ; dans la pratique, les méthodes d’évaluation sont présentes à chaque étape de la programmation d’un modèle si bien que presque aucun choix ne devrait être pris sans validation par une métrique ou une autre. Comme nous nous sommes concentré·es sur deux types d’algorithmes, nous n’aborderons que les méthodes d’évaluation pour les tâches de régression et de classification.

## Mesurer une erreur

Par métrique, on entend une façon d’évaluer la qualité d’une prédiction par mesure de la distance entre la réalité observée et la valeur calculée par un algorithme. Si l’on souhaite par exemple prédire la note d’un élève au prochain examen de français en se basant uniquement sur sa moyenne dans la matière – disons 12 –, l’algorithme de prédiction vaudra simplement :

$$\hat{y} = \mu$$

L’élève obtient finalement une note de 11. Pour mesurer l’erreur de la prédiction, il suffit de soustraire $\hat{y}$ de $y$, soit un résultat de $-1$. Remarquons que si sa note avait été de 13, le résultat aurait été positif : $13 - 12 = 1$. Or, $-1$  et $+1$ étant situés à égales distances de la prédiction, ils représentent la même réalité géométrique. Dans les deux cas, l’erreur est réputée être de $1$. On utilise donc plutôt une formule impliquant les valeurs absolues :

$$e = \lvert y - \hat{y}\rvert$$

## Métriques pour les tâches de régression

Dans l’exemple de l’introduction, il s’agissait simplement de calculer l’erreur pour un couple unique prédiction/résultat. Qu’en serait-il si nous avions une série de prédictions et une série de résultats ? Plutôt que de calculer indépendamment les erreurs de chaque prédiction, nous préférerions obtenir une mesure de l’ensemble.

Et pour corser le tout, il existe plusieurs métriques qui ne répondent pas tout à fait aux mêmes enjeux. Choisir la plus adaptée à la situation peut ainsi devenir une nécessité pour ajuster plus finement encore le modèle.

Prenons le cas fictif de la pluviométrie au-dessus de la commune de Pont-Aven avec d’un côté les précipitations mesurées en millimètres pour les mois de janvier à mai 2022 et, de l’autre, des prédictions imaginaires :

In [ ]:
import pandas as pd
import seaborn as sns

# series
series = {
    "months": ["Jan", "Feb", "March", "April", "May"],
    "rainfall": [70, 65, 55, 50, 9],
    "predictions": [35, 60, 75, 45, 20]
}
# dataframe
df = pd.DataFrame(series)

# column 'months' still an id var, while two others are registered in a col 'Measure'
df2 = pd.melt(df, id_vars="months", var_name="Measure", value_name="mm")

# graph
_ = sns.lineplot(data=df2, x="months", y="mm", hue="Measure", marker="o")
sns.despine()

### Le coefficient de détermination linéaire de Pearson ($R^2$)

Le $R^2$ est un score qui mesure la qualité de la prédiction d’un modèle de régression linéaire en évaluant la variance d’une variable par rapport à une autre variable. Il est défini par la relation suivante pour un résultat généralement compris dans l’intervalle $[0,1]$ :

$$R^2 = 1 - \frac{\sum_{i=1}^k(y_i - \hat{y}_i)^2}{\sum_{i=1}^k(y_i - \bar{y})^2}$$

Son analyse est très intuitive mais elle implique deux critères :
- le modèle est linéaire ;
- une seule variable explicative est concernée.

Un $R^2$ de 1.0 est un score parfait quand un score de 0.0 indiquerait que le modèle prédit toujours la valeur attendue (la moyenne). Un score négatif reste possible mais serait révélateur d’une erreur de méthodologie (données arbitrairement mauvaises).

Dans le cas de notre exemple, la prédiction n’est clairement pas linéaire, aussi le calcul du $R^2$ ne devrait pas servir pour l’évaluation de notre modèle. À titre d’exercice, voyons ce qu’il donne en invoquant la fonction `r2_score()` du module `metrics` de *Scikit-learn* :

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(df.rainfall, df.predictions)
r2

### L’erreur quadratique moyenne (MSE)

La MSE (*mean square error*) et sa cousine, la RMSE (*root mean square error*), sont les deux métriques les plus couramment utilisées en *machine learning*. La MSE calcule la moyenne des carrés des erreurs selon la formule :

$$\text{MSE} = \frac{1}{k}\sum_{i=0}^{k-1}(y_i - \hat{y}_i)^2$$

Comme entre en jeu un calcul au carré, la MSE pénalise plus fortement les grandes erreurs et, dans le même ordre d’idée, sera très sensible aux données aberrantes (*outliers*). La fonction dans *Scikit-learn* est `mean_squared_error()` :

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(df.rainfall, df.predictions)
mse

### La racine de l’erreur quadratique moyenne (RMSE)

Plus facile à interpréter que la MSE, la RMSE (*root mean square error*) s’exprime dans l’unité de la variable à prédire en extrayant la racine carrée de la MSE :

$$\text{RMSE} = \sqrt{ \frac{1}{k}\sum_{i=0}^{k-1}(y_i - \hat{y}_i)^2 }$$

À noter qu’elle souffre des mêmes limites que la MSE : une grande sensibilité aux *outliers* ainsi qu’une incidence forte sur les erreurs importantes. Pour la calculer avec *Scikit-learn*, il suffit de prendre la racine carrée de la MSE :

In [ ]:
# power of 0.5 = square root
rmse = mse ** 0.5
rmse

### L’erreur absolue moyenne (MAE)

Quand les valeurs extrêmes d’un jeu de données sont quantitativement importantes, la RMSE pourrait conduire à des erreurs d’interprétation. Dans un tel cas de figure, la MAE (*mean absolute error*) peut lui être préférée : en calculant la moyenne de valeurs absolues, elle ne pénalise plus autant les grandes erreurs et se rend moins sensible aux données aberrantes. La formule de la MAE vaut ainsi :

$$\text{MAE} = \frac{1}{k}\sum_{i=0}^{k-1} \lvert y_i - \hat{y}_i\rvert$$

Dans *Scikit-learn*, la fonction `mean_absolute_error()` se charge du calcul :

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(df.rainfall, df.predictions)
mae

## Métriques pour les tâches de classification

Dans la grande famille des tâches de classification, nous reconnaissons trois catégories :
- **la classification binaire** (l’observation appartient-elle à la classe cible ou non ?) ;
- **la classification multi-classes** (parmi toutes, à quelle classe l’observation appartient-elle ?) ;
- **la classification multi-étiquettes** (l’observation appartient-elle à plusieurs classes ?).

Là encore, selon la nature de la tâche, nous ne choisirons pas forcément la même métrique.

### Mesures générales de performance

Évaluer les performances d’un classificateur étant plus complexe que pour les tâches de régression, il est nécessaire d’aborder en premier lieu certaines généralités.

#### L’exactitude (*accuracy*)

La toute première est de ne jamais oublier qu’un très mauvais classificateur peut obtenir des résultats stupéfiants. Prenons l’exemple d’un jeu de données factice qui comporte cent observations étiquetées selon deux modalités : *chat* ou *pas chat*.

In [ ]:
from random import shuffle

# 5 out of an hundred are cats
dataset = [
    "cat" if i < 5 else "not cat"
    for i in range(100)
]
# every day I'm shufflin
shuffle(dataset)

Générons une autre liste pour les prédictions avec la seule étiquette *not cat* :

In [ ]:
predictions = [ "not cat" for i in range(100) ]

Et mesurons la performance de notre algorithme prédictif grâce au score d’exactitude (*accuracy*) :

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(dataset, predictions)

**95 % !** Un taux d’exactitude à faire pâlir les diseuses de bonne aventure, non ? Pour cette raison, on ne se satisfera jamais du score d’exactitude, quitte même à s’en méfier dès que les jeux de données sont asymétriques.

#### Une matrice de confusion

La matrice de confusion repose sur un principe simple : compter le nombre de fois où les observations ont été bien ou mal étiquetées. Elle révèle ainsi quatre informations essentielles :
- **les vrais positifs** (*true positive*), le classificateur a repéré qu’il s’agissait d’un chat ;
- **les vrais négatifs** (*true negative*), le classificateur a repéré qu’il ne s’agissait pas d’un chat ;
- **les faux positifs** (*false positive*), le classificateur a cru qu’il s’agissait d’un chat ;
- **les faux négatifs** (*false negative*), le classificateur aurait dû voir qu’il s’agissait d’un chat.

Préparons un jeu de données aléatoire avec la fonction `make_classification()` du module `sklearn.datasets` et effectuons des prédictions à partir d’un modèle de classification naïve bayésienne :

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# make dataset
X, y = make_classification(n_samples=1000, n_classes=2, n_features=5, n_informative=3, random_state=42)

# make train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# model
model = GaussianNB()
model.fit(X_train, y_train)

# predictions
y_pred = model.predict(X_test)

La matrice de confusion s’obtient avec la fonction `confusion_matrix()` du module `sklearn.metrics` :

In [ ]:
from sklearn.metrics import confusion_matrix

cfm = confusion_matrix(y_test, y_pred)

Chaque ligne correspond à une classe réelle et chaque colonne à une classe prédite avec, sur la première ligne, la classe négative et, sur la seconde, la classe positive, tel que dans le tableau suivant :

|prédites/réelles|Classe négative|Classe positive|
|-|:-:|:-:|
|Classe négative|TN (85)|FP (9)|
|Classe positive|FN (3)|TP (103)|

Une méthode `.ravel()` permet de récupérer chacun de ces indicateurs :

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

Notons également l’existence d’une classe `ConfusionMatrixDisplay` pour afficher la matrice de confusion :

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

display = ConfusionMatrixDisplay(confusion_matrix=cfm, display_labels=model.classes_)
_ = display.plot()

En guise de conclusion, signalons que cette matrice de confusion détermine plusieurs scores : la précision, la sensibilité (ou rappel) et le score $F_1$.

### Classification binaire

Dans le cas d’un projet basé sur un classificateur binaire, on fera toujours appel à une matrice de confusion à partir de laquelle on obtiendra d’autres métriques plus parlantes.

#### La précision

La précision s’intéresse à l’exactitude des prédictions positives. Elle se calcule en effectuant le rapport entre les vrais positifs – les fois où le classificateur a correctement étiqueté la classe positive, et la somme de toutes les prédictions positives – incluant donc les fois où le classificateur s’est trompé. La formule vaut :

$$\text{precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}$$

Soit, pour notre exemple à partir du classificateur naïf bayésien :

In [ ]:
tp / (tp + fp)

Concrètement, la précision signifie que notre classificateur a raison dans 91,96 % des cas quand il prédit la classe positive (1).

Il est à noté que *Scikit-Learn* met à disposition une fonction `precision_score()` dans le module `metrics` :

In [ ]:
from sklearn.metrics import precision_score

precision_score(y_test, y_pred)

#### Le rappel

La métrique de précision d’un classificateur s’accompagne toujours du rappel (*recall*), ou sensibilité, qui détermine le taux de classes positives qu’il a correctement étiquetées. Elle s’obtient avec l’équation :

$$\text{recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$$

Dans notre exemple :

In [ ]:
tp / (tp + fn)

En termes plus parlants, notre classificateur parvient à détecter correctement 97,17 % des observations de la classe positive (1).

Comme pour la précision, il existe une fonction dans *Scikit-Learn* pour calculer directement le rappel :

In [ ]:
from sklearn.metrics import recall_score

recall_score(y_test, y_pred)

#### Le score $F_1$

Entraînons un nouveau classificateur, basé cette fois-ci sur un arbre de décision :

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# predictions
y_pred = model.predict(X_test)

Examinons sa matrice de confusion :

In [ ]:
cfm = confusion_matrix(y_test, y_pred)
_ = ConfusionMatrixDisplay(confusion_matrix=cfm, display_labels=model.classes_).plot()

Et ressortons les métriques de précision et de rappel :

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

precision = tp / (tp + fp)

recall = tp / (tp + fn)

print(
    f"Précision : {precision:.2%}",
    f"Rappel : {recall:.2%}",
    sep="\n"
)

Dans cet exemple, la précision de l’arbre de décision est supérieure à celle du classificateur naïf bayésien alors que son rappel est inférieur. Lequel des deux choisir ? Pour aider à la décision, il existe une autre métrique, le score $F_1$, qui effectue la moyenne harmonique entre les deux :

$$F_1 = 2 \times \frac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}}$$

La moyenne harmonique donnant plus d’important aux valeurs faibles, un classificateur ne pourra être bien noté par cette métrique que si sa précision et son rappel sont élevés.

In [ ]:
F1_bayes = 2 * ((91.96 * 97.17) / (91.96 + 96.17))
F1_tree = 2 * ((94.39 * 95.28) / (94.39 + 95.28))

print(
    f"Score F1 du classificateur naïf bayésien : {F1_bayes:.2f} %",
    f"Score F1 de l’arbre de décision : {F1_tree:.2f} %",
    sep="\n"
)

Une fonction de *Scikit-Learn* permet d’obtenir directement le résultat :

In [ ]:
from sklearn.metrics import f1_score

F1_tree = f1_score(y_test, y_pred)

print(f"Score F1 de l’arbre de décision : {F1_tree:.2%}")

Dans cet exemple, toutes choses étant égales par ailleurs, l’arbre de décision semble fournir le meilleur compromis précision/rappel. Pour autant, l’objet de votre étude lui-même pourrait dicter de privilégier l’une ou l’autre.

Si nous élaborons un classificateur de spams, nous préférerons sans doute un modèle avec un fort rappel (aucun spam ne passe) même si sa précision laisse à désirer (certains mails sont faussement identifiés comme spams). À l’inverse, si nous cherchons un classificateur qui épargne nos oreilles de chansons nocives, nous serons prêt·es à quelques sacrifices : de bonnes chansons seront écartées mais au moins nous serons préservé·es de la soupe qu’écoute la jeunesse actuelle !

#### La courbe ROC

Une autre métrique couramment utilisée afin d’évaluer la performance d’un classificateur binaire est la courbe ROC (*Receiver Operating Characteristic*) ou courbe d’efficacité du récepteur. Cela consiste à croiser le taux de vrais positifs avec le taux de faux positifs (le rappel) et d’observer l’aire sous la courbe. Un modèle purement aléatoire serait proche de 0,5 quand un modèle parfait afficherait 1.

Afin d’en comprendre le fonctionnement, entraînons un nouveau modèle, basé sur une régression logistique :

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="liblinear", random_state=42)
_ = model.fit(X_train, y_train)

Grâce à la fonction `cross_val_predict()`, nous effectuons une validation croisée à $K$ passes qui nous renvoie les prédictions pour chaque bloc de test. L’argument `method` fixé à `predict_proba` permet de ressortir des probabilités pour chaque classe plutôt que la classe prédite :

In [ ]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(model, X_train, y_train, cv=3, method="predict_proba")

Récupérons à présent les scores pour la classe positive :

In [ ]:
y_scores = y_pred[:, 1]

*Scikit-Learn* donne accès aux fonctions `roc_curve()` et `roc_auc_score()` pour, dans l’ordre, obtenir la courbe ROC et l’aire sous la courbe :

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

roc_auc = roc_auc_score(y_train, y_scores)

fpr, tpr, thresholds = roc_curve(y_train, y_scores)

Il est ensuite possible d’afficher le résultat avec, en pointillés, ce que donnerait un classificateur purement aléatoire :

In [ ]:
import matplotlib.pyplot as plt

_ = sns.lineplot(x=fpr, y=tpr)

plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title(f"ROC AUC : {roc_auc:.4f }")
plt.plot([0, 1], [0, 1], 'k--', alpha=.2)

sns.despine()

#### Le compromis précision/rappel

La courbe ROC aura parfois la mauvaise tendance à nous faire croire que notre classificateur est parfaitement paramétré. Une autre courbe, nommée PR pour *precision-recall*, est à préférer si la classe positive est rare ou si l’on attache plus d’importance aux fois où le classificateur s’est trompé (les faux positifs et négatifs).

Le module `sklearn.metrics` expose une fonction `precision_recall_curve()` qui renvoie les scores de précision et de rappel en fonction de seuils (ici, des probabilités) :

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores)

Croisons ensuite les précisions et les rappels :

In [ ]:
_ = sns.lineplot(x=recalls, y=precisions)

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("PR curve")
plt.plot([0.85, 0.85], [0.94, 0.5], 'k--', alpha=.2)
plt.plot([0, 0.85], [0.94, 0.94], 'k--', alpha=.2)
plt.text(0.87, 0.97, "P : 94 %")
plt.text(0.87, 0.95, "R : 85 %")

sns.despine()

On observe sur ce graphique que la précision chute après 85 % de rappel. Si nous voulons une meilleure précision, elle se fera au détriment du rappel et inversement.

Il peut aussi être utile de représenter la précision et le rappel en fonction du seuil :

In [ ]:
_ = sns.lineplot(x=thresholds, y=precisions[:-1], label="Precision")
_ = sns.lineplot(x=thresholds, y=recalls[:-1], label="Recall")

plt.plot([0.63, 0.63], [0, 0.94], 'k--', alpha=.2)
plt.text(0.64, 0.98, "P : 94 %")
plt.text(0.64, 0.86, "R : 85 %")

plt.xlabel("Threshold")

sns.despine()

La projection de la ligne pointillée révèle le seuil à choisir pour obtenir cette précision de 94 %. Plus finement, nous pouvons sélectionner les indices qui correspondent à cette contrainte avec la fonction `argmax()` de *Numpy* qui renvoie l’indice du premier élément qui respecte la condition de précision :

In [ ]:
import numpy as np

i_94 = np.argmax(precisions >= 0.94)

À présent, nous pouvons sélectionner le seuil le plus bas qui réponde à la précision :

In [ ]:
t_94 = thresholds[i_94]

print(f"Seuil de probabilité pour une précision de 94 % : { t_94 }")

Plus haut, nous avons appelé la fonction `cross_val_predict()` pour obtenir dans une variable `y_scores` des prédictions sur le jeu d’entraînement. Repérons les observations qui ont une probabilité au-dessus du seuil qui répond à la précision de 94 % :

In [ ]:
y_train_94 = (y_scores >= t_94)

Vérifions pour se rassurer que les valeurs de précision et de rappel correspondent :

In [ ]:
p = precision_score(y_train, y_train_94)
r = recall_score(y_train, y_train_94)
f1 = f1_score(y_train, y_train_94)

print(
    f"Précision : {p:.2%}",
    f"Rappel : {r:.2%}",
    f"F1 score : {f1:.2%}",
    sep="\n"
)

Grâce à ces manipulations, il est assez aisé de paramétrer un classificateur avec un certain taux de précision. Attention toutefois à ne jamais négliger le rappel ! Dans notre exemple, le F1 score est descendu en dessous des 90 %.

### Classification multi-classes

Lorsqu’un classificateur binaire prédit l’appartenance d’une observation à une classe ou à une autre, un classificateur multi-classes détermine laquelle, parmi toutes, a sa préférence. Et, pour réaliser une telle tâche, il suffit d’entraîner plusieurs classificateurs binaires.

Mettons que nous entraînons une machine pour détecter si telle musique appartient au mouvement punk, au genre métal, au grunge ou encore au hardcore. Deux stratégies se présentent :

- OvA (*One versus All*), qui consiste à décider avec un premier classificateur si la musique en question appartient à la première classe (punk) ou non, puis, avec un deuxième classificateur, si c’est du métal ou non, etc.
- OvO (*One versus One*), qui consiste à entraîner $\frac{n(n-1)}{2}$ classificateurs, soit 6 pour notre objet d’étude fictif :
    - punk ou métal ?
    - punk ou grunge ?
    - punk ou hardcore ?
    - métal ou grunge ?
    - métal ou hardcore ?
    - grunge ou hardcore ?

On peut penser intuitivement qu’il est coûteux d’entraîner de nombreux classificateurs plutôt qu’un seul lorsque les classes sont multiples, mais certains algorithmes comme les SVM manquent de performance face à des jeux d’entraînement de grande taille. Dans ce cas, la stratégie à privilégier sera clairement la OvO quand, dans la plupart des autres, on préférera la OvA.

À cette subtilité, il convient de rajouter que certains algorithmes gèrent nativement la classification multi-classes (les forêts aléatoires ou la classification naïve bayésienne) quand d’autres sont exclusivement binaires (les SVM ou la régression logistique). Heureusement, si l’on invoque l’un de ces derniers algorithmes pour une tâche multi-classes, *Scikit-Learn* le comprend imédiatement et applique l’une des deux stratégies. Par exemple, avec un SVM, il utilisera une stratégie OvO.

Entraînons un classificateur SVM à partir d’un jeu de données avec quatre modalités :

In [ ]:
from sklearn.svm import SVC

# make dataset
X, y = make_classification(n_samples=10000, n_classes=4, n_features=5, n_informative=3, random_state=42)

# make train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# model
model = SVC()
_ = model.fit(X_train, y_train)

Prenons la première observation du jeu de test afin d’effectuer une prédiction :

In [ ]:
model.predict([ X_test[0] ])

Les algorithmes basés sur une fonction de décision exposent une méthode `.decision_function()` afin de consulter les scores attribués à chaque modalité :

In [ ]:
# how the decision is made?
model.decision_function([ X_test[0] ])

Le score le plus haut est bien attribué à la dernière classe, positionnée à l’indice 3. Pour s’assurer que l’indice 3 correspond bien à la classe étiquetée *3*, ne pas oublier de consulter la propriété `.classes_` :

In [ ]:
# classes are ordered, could have been other way!
model.classes_

À noter que pour obliger un algorithme à adopter une stratégie qui n’est pas conforme à sa nature, il existe des classes spécifiques `OneVsRestClassifier` et `OneVsOneClassifier` :

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(SVC(random_state=42))
_ = model.fit(X_train, y_train)

model.predict([ X_test[0] ])

La propriété `estimators_` permet de vérifier que la stratégie OvA utilise bien autant d’estimateurs que de modalités :

In [ ]:
len(model.estimators_)

#### Analyser la matrice de confusion

Comme pour la classification binaire, le premier réflexe est de s’attarder sur la matrice de confusion. Commençons par obtenir des prédictions grâce à la validation croisée :

In [ ]:
# custom class names
model.classes_ = ["punk", "métal", "grunge", "hardcore"]

y_pred = cross_val_predict(model, X_test, y_test, cv=3)

Affichons à présent la matrice de confusion :

In [ ]:
cfm = confusion_matrix(y_test, y_pred)
_ = ConfusionMatrixDisplay(confusion_matrix=cfm, display_labels=model.classes_).plot()

Les scores les plus hauts étant sur la diagonale, d’un coup d’œil nous savons que notre classificateur est plutôt performant. Une hésitation toufois pour le grunge, qui semble moins facilement identifiable que les autres styles musicaux : 65 ont été considérés comme du punk.

Notre machine n’est finalement pas si performante que cela ou bien notre jeu de données comporte-t-il moins de musiques de ce style que les autres ?

Faisons le total pour chaque genre :

In [ ]:
row_sums = cfm.sum(axis=1, keepdims=True)
row_sums

En divisant les deux matrices, nous calculons le taux d’erreur pour chaque modalité :

In [ ]:
error_rates = cfm / row_sums

Avant d’afficher de nouveau une carte thermique, remplissons la diagonale de zéros pour plus de lisibilité :

In [ ]:
np.fill_diagonal(error_rates, 0)
_ = sns.heatmap(error_rates, cmap="Greys")

La carte se lit dans les deux sens :

- en lignes, les classes véritables ;
- en colonnes, les classes prédites.

La ligne 2, comme elle est plus sombre, montre que le grunge est souvent mal étiqueté, quand le style punk (ligne 0) semble être lui bien identifié. La colonne 3, étant plus claire, tend à indiquer que de nombreuses musiques lui sont faussement attribuées.

De l’analyse de la matrice de confusion, il est souhaitable de tirer des pistes d’amélioration, comme, dans notre exemple, enrichir le jeu de données de musiques hardcore ou se poser la question de son échec à classer les morceaux punks.